In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from pandas import DataFrame as DF
import time
from pdf2str import extract_text_from_pdf
import os
import math

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [2]:
def click_button(xpath='',sleep=0):
    button=browser.find_element_by_xpath(xpath)
    button.click()
    time.sleep(sleep)
    
def sendKeys(keys='',xpath='',sleep=0):
    search_txt=browser.find_element_by_xpath(xpath)
    search_txt.send_keys(keys)
    time.sleep(sleep)
    


In [3]:
# url='https://scholar.google.com/citations?user=f6SBzrAAAAAJ&hl=zh-TW&oi=ao'
# response = rq.get(url) 
# html_doc = response.text 
# soup = BeautifulSoup(response.text) 
# # a=soup.find('ul',{'class':'link-list-aroow-wide column2'}).find_all('a')
# # prefix='https://www.2020games.metro.tokyo.lg.jp/eng/taikaijyunbi/taikai/kaijyou/'
# # sites=[prefix+x['href'] for x in a]
# soup

In [4]:
options = Options()
options.headless = False

default_content_setting_values={'notifications' : 2}
prefs = {'profile.default_content_setting_values':default_content_setting_values}
options.add_experimental_option('prefs',prefs)

browser=webdriver.Chrome('/Users/ryh/Desktop/crawler/chromedriver',options=options)
scholar_url='https://scholar.google.com/citations?user=zJ3GT9IAAAAJ&hl=zh-TW&oi=sra'
browser.get(scholar_url)


In [5]:
soup = BeautifulSoup(browser.page_source)
scholar_name=soup.find('div',{'id':"gsc_prf_in"}).text
scholar_name=scholar_name.replace(' ','-')
scholar_name


'Ming-Chuan-Yang'

In [6]:
# page_source = browser.page_source
# soup = BeautifulSoup(page_source)
# a=soup.find('button',{'class':'gs_btnPD gs_in_ib gs_btn_flat gs_btn_lrge gs_btn_lsu'})
# is_more='disabled' not in str(a)
# is_more

In [7]:
def is_more(browser):
    page_source = browser.page_source
    soup = BeautifulSoup(page_source)
    a=soup.find('button',{'class':'gs_btnPD gs_in_ib gs_btn_flat gs_btn_lrge gs_btn_lsu'})
    is_more='disabled' not in str(a)
    return is_more

In [8]:
# more_button=browser.find_element_by_xpath("//*[contains(text(), '顯示更多結果')]")
# more_button.click()

In [9]:
# click more while is_more
while True:
    if is_more(browser)==False: break
    more_button=browser.find_element_by_xpath("//*[contains(text(), '顯示更多結果')]")    
    time.sleep(3)
    more_button.click()
    time.sleep(1)

In [10]:
soup = BeautifulSoup(browser.page_source)
paper_num=soup.find('div',{'id':'gsc_lwp'}).find('span',{'id':"gsc_a_nn"}).text.split('–')[-1]
paper_num=int(paper_num)
paper_num


10

# one paper

In [11]:
# i=1
# paper_button=browser.find_element_by_xpath('//*[@id="gsc_a_b"]/tr[%s]/td[1]/a'%(i))
# paper_button.click()

In [12]:
# soup = BeautifulSoup(browser.page_source)
# a=soup.find('a',{'class':'gsc_vcd_title_link'})
# paper_name=a.text
# paper_url=a['href']
# paper_name,paper_url

In [13]:
# paper_conference=soup.find("div",{'class':'gsc_vcd_field'}, string="會議")
# paper_conference=paper_conference.find_next('div').text if paper_conference !=None else 'x'
# paper_conference

def title2content(soup,title=''):
    paper_conference=soup.find("div",{'class':'gsc_vcd_field'}, string=title)
    
    if title=='引文總數':
        paper_conference=int(paper_conference.find_next('a').text.split(' ')[1]) if paper_conference !=None else 0
        return paper_conference
    
    paper_conference=paper_conference.find_next('div').text if paper_conference !=None else 'x'
    return paper_conference


# title2content(soup,title='引文總數')

In [14]:
titles=['作者','發布日期','描述','引文總數','會議','期刊']
# for title in titles:
#     content=title2content(soup,title=title)
#     print(title,content)

In [15]:
# pdf_url=soup.find_all("div",{'class':'gsc_vcd_title_ggi'})
# pdf_url=[x.find('a')['href'] for x in pdf_url] if len(pdf_url)!=0 else 'x'
# pdf_url

In [16]:
# row=DF([[paper_name,paper_url]+[title2content(soup,title=title) for title in titles]])
# row.columns=['paper_name','paper_url']+titles
# row

In [17]:
def get_i_paper(result,i):
    paper_button=browser.find_element_by_xpath('//*[@id="gsc_a_b"]/tr[%s]/td[1]/a'%(i))
    paper_button.click()
    time.sleep(1)

    soup = BeautifulSoup(browser.page_source)

    a=soup.find('a',{'class':'gsc_vcd_title_link'})
    paper_name=a.text
    paper_url=a['href']

    pdf_url=soup.find_all("div",{'class':'gsc_vcd_title_ggi'})
    pdf_url=[x.find('a')['href'] for x in pdf_url if 'pdf' in x.find('a')['href']] if len(pdf_url)!=0 else 'x'
    pdf_url=pdf_url[0] if len(pdf_url)!=0 else 'x'

    row=[[paper_name,paper_url]+[title2content(soup,title=title) for title in titles]+[pdf_url]]
    row=DF(row)
    row.columns=['paper_name','paper_url']+titles+['pdf_url']
    result=result.append(row)

    x_button=browser.find_element_by_xpath('//*[@id="gs_md_cita-d-x"]')
    x_button.click()
    time.sleep(1)
    
    return result
    

In [18]:
result=DF()

In [19]:
i_list=list(range(1,paper_num+1))
while True:
    print(' --------------------- ')
    print('len(i_list)',len(i_list))
    print('result.shape',result.shape)
    
    if len(i_list)==0: break
    i=i_list.pop(0)
    print('i',i)
    try:
        result=get_i_paper(result,i)
    except Exception as e:
        print(e)
        print('error',i)
        i_list.append(i)

 --------------------- 
len(i_list) 10
result.shape (0, 0)
i 1
 --------------------- 
len(i_list) 9
result.shape (1, 9)
i 2
 --------------------- 
len(i_list) 8
result.shape (2, 9)
i 3
 --------------------- 
len(i_list) 7
result.shape (3, 9)
i 4
 --------------------- 
len(i_list) 6
result.shape (4, 9)
i 5
 --------------------- 
len(i_list) 5
result.shape (5, 9)
i 6
 --------------------- 
len(i_list) 4
result.shape (6, 9)
i 7
 --------------------- 
len(i_list) 3
result.shape (7, 9)
i 8
 --------------------- 
len(i_list) 2
result.shape (8, 9)
i 9
 --------------------- 
len(i_list) 1
result.shape (9, 9)
i 10
 --------------------- 
len(i_list) 0
result.shape (10, 9)


In [20]:
# result.loc[result['引文總數']=='x','引文總數']=0
result=result.reset_index(drop=True)
file_name='%s.pickle'%(scholar_name)
path=os.path.join('.','scholar',file_name)
path

'./scholar/Ming-Chuan-Yang.pickle'

In [21]:
result.to_pickle(path)

# download pdf


In [22]:
result=pd.read_pickle(path)
result['pdf_path']='x'
result

,paper_name,paper_url,作者,發布日期,描述,引文總數,會議,期刊,pdf_url,pdf_path
0,On the inapproximability of maximum intersecti...,https://www.sciencedirect.com/science/article/...,"Min-Zheng Shieh, Shi-Chun Tsai, Ming-Chuan Yang",2012/10/15,"Given u sets, we want to choose exactly k sets...",14,x,Information Processing Letters,https://ir.nctu.edu.tw/bitstream/11536/16807/1...,x
1,Detecting amplification attacks with software ...,https://ieeexplore.ieee.org/abstract/document/...,"Chih-Chieh Chen, Yi-Ren Chen, Wei-Chih Lu, Shi...",2017/8/7,Distributed denial of service (DDoS) is an att...,7,2017 IEEE conference on dependable and secure ...,x,x,x
2,Restrictions of nondegenerate Boolean function...,https://ieeexplore.ieee.org/abstract/document/...,"Chia-Jung Lee, Satyanarayana V Lokam, Shi-Chun...",2015/6/14,"A Boolean function f : {0, 1} n → {0, 1} is ca...",3,2015 IEEE International Symposium on Informati...,x,x,x
3,Online prediction problems with variation,https://link.springer.com/chapter/10.1007/978-...,"Chia-Jung Lee, Shi-Chun Tsai, Ming-Chuan Yang",2014/8/4,We study the prediction with expert advice pr...,1,International Computing and Combinatorics Conf...,x,x,x
4,PM2. 5 Forecasting Using Pre-trained Components,https://ieeexplore.ieee.org/abstract/document/...,"Ming-Chuan Yang, Meng Chang Chen",2018/12/10,This work investigates the efforts of composit...,0,2018 IEEE International Conference on Big Data...,x,x,x
5,Wrapped Loss Function for Regularizing Nonconf...,https://arxiv.org/abs/1808.06733,"Chun Ting Liu, Ming Chuan Yang, Meng Chang Chen",2018/8/21,Multi-output is essential in machine learning ...,0,x,arXiv preprint arXiv:1808.06733,https://arxiv.org/pdf/1808.06733,x
6,Representing Symmetric Boolean Functions with ...,http://search.ebscohost.com/login.aspx?direct=...,"Shi-Chun Tsai, Ming-Chuan Yang",2018/1,Polynomial degree is an important measure for ...,0,x,Journal of Information Science and Engineering,x,x
7,Forwarding path discovery with software define...,https://ieeexplore.ieee.org/abstract/document/...,"Chih-Chieh Chen, Yi-Ren Chen, Shi-Chun Tsai, M...",2017/9/27,"Without STP and LLDP, we propose a new approac...",0,2017 19th Asia-Pacific Network Operations and ...,x,x,x
8,On Restricting No-Junta Boolean Function and D...,https://arxiv.org/abs/1502.00357,"Chia-Jung Lee, Satya V Lokam, Shi-Chun Tsai, M...",2015/2/2,Let be the set of Boolean functions depending...,0,x,arXiv preprint arXiv:1502.00357,https://arxiv.org/pdf/1502.00357,x
9,布林函數多項式表示法之次方下界,http://www.airitilibrary.com/Publication/alDet...,Ming-Chuan Yang,2015/1/1,Polynomial degree is an important measure for ...,0,x,交通大學資訊科學與工程研究所學位論文,x,x


In [23]:
def download_i_paper(result,i):
    row=result.loc[i,:]
    pdf_url=row['pdf_url']
        
    paper_name=row['paper_name']
    for c in [',',':']:
        paper_name=paper_name.replace(c,'')
    paper_name=paper_name.replace(' ','-')
    pdf_path='./pdf/%s.pdf'%(paper_name)
    
    if os.path.isfile(pdf_path)!=True:
        myfile = rq.get(pdf_url)
        with open(pdf_path, 'wb') as f:
            f.write(myfile.content)
        
    result.loc[i,'pdf_path']=pdf_path
    return result


In [33]:
i_list=result[result.pdf_url!='x'].index.tolist()
len(i_list)

3

In [25]:
while True:
    print(' --------------------- ')
    print('len(i_list)',len(i_list))
    
    if len(i_list)==0: break
    i=i_list.pop(0)
    print('i',i)
    try:
        result=download_i_paper(result,i)
    except Exception as e:
        print(e)
        print('error',i)
        i_list.append(i)






 --------------------- 
len(i_list) 3
i 0
 --------------------- 
len(i_list) 2
i 5
 --------------------- 
len(i_list) 1
i 8
 --------------------- 
len(i_list) 0


In [26]:
result.to_pickle(path)

# check keywords


In [27]:
result=pd.read_pickle(path)
result['keywords']='x'
result

,paper_name,paper_url,作者,發布日期,描述,引文總數,會議,期刊,pdf_url,pdf_path,keywords
0,On the inapproximability of maximum intersecti...,https://www.sciencedirect.com/science/article/...,"Min-Zheng Shieh, Shi-Chun Tsai, Ming-Chuan Yang",2012/10/15,"Given u sets, we want to choose exactly k sets...",14,x,Information Processing Letters,https://ir.nctu.edu.tw/bitstream/11536/16807/1...,./pdf/On-the-inapproximability-of-maximum-inte...,x
1,Detecting amplification attacks with software ...,https://ieeexplore.ieee.org/abstract/document/...,"Chih-Chieh Chen, Yi-Ren Chen, Wei-Chih Lu, Shi...",2017/8/7,Distributed denial of service (DDoS) is an att...,7,2017 IEEE conference on dependable and secure ...,x,x,x,x
2,Restrictions of nondegenerate Boolean function...,https://ieeexplore.ieee.org/abstract/document/...,"Chia-Jung Lee, Satyanarayana V Lokam, Shi-Chun...",2015/6/14,"A Boolean function f : {0, 1} n → {0, 1} is ca...",3,2015 IEEE International Symposium on Informati...,x,x,x,x
3,Online prediction problems with variation,https://link.springer.com/chapter/10.1007/978-...,"Chia-Jung Lee, Shi-Chun Tsai, Ming-Chuan Yang",2014/8/4,We study the prediction with expert advice pr...,1,International Computing and Combinatorics Conf...,x,x,x,x
4,PM2. 5 Forecasting Using Pre-trained Components,https://ieeexplore.ieee.org/abstract/document/...,"Ming-Chuan Yang, Meng Chang Chen",2018/12/10,This work investigates the efforts of composit...,0,2018 IEEE International Conference on Big Data...,x,x,x,x
5,Wrapped Loss Function for Regularizing Nonconf...,https://arxiv.org/abs/1808.06733,"Chun Ting Liu, Ming Chuan Yang, Meng Chang Chen",2018/8/21,Multi-output is essential in machine learning ...,0,x,arXiv preprint arXiv:1808.06733,https://arxiv.org/pdf/1808.06733,./pdf/Wrapped-Loss-Function-for-Regularizing-N...,x
6,Representing Symmetric Boolean Functions with ...,http://search.ebscohost.com/login.aspx?direct=...,"Shi-Chun Tsai, Ming-Chuan Yang",2018/1,Polynomial degree is an important measure for ...,0,x,Journal of Information Science and Engineering,x,x,x
7,Forwarding path discovery with software define...,https://ieeexplore.ieee.org/abstract/document/...,"Chih-Chieh Chen, Yi-Ren Chen, Shi-Chun Tsai, M...",2017/9/27,"Without STP and LLDP, we propose a new approac...",0,2017 19th Asia-Pacific Network Operations and ...,x,x,x,x
8,On Restricting No-Junta Boolean Function and D...,https://arxiv.org/abs/1502.00357,"Chia-Jung Lee, Satya V Lokam, Shi-Chun Tsai, M...",2015/2/2,Let be the set of Boolean functions depending...,0,x,arXiv preprint arXiv:1502.00357,https://arxiv.org/pdf/1502.00357,./pdf/On-Restricting-No-Junta-Boolean-Function...,x
9,布林函數多項式表示法之次方下界,http://www.airitilibrary.com/Publication/alDet...,Ming-Chuan Yang,2015/1/1,Polynomial degree is an important measure for ...,0,x,交通大學資訊科學與工程研究所學位論文,x,x,x


In [28]:
i_list=result[result.pdf_path!='x'].index.tolist()
len(i_list)

3

In [36]:
def check_i_paper(result,i):
    keywords=['knn','kriging','interpolation','transfer']
    ks=[]
    row=result.loc[i,:]
    pdf_path=row['pdf_path']
    paper_text=extract_text_from_pdf(pdf_path)
    paper_text=paper_text.lower()
    for keyword in keywords:
        if keyword in paper_text: 
            ks.append(keyword)
    
    ks=','.join(ks) if len(ks)>=1 else 'no'
    result.loc[i,'keywords']=ks
    return result            

In [30]:
while True:
    print(' --------------------- ')
    print('len(i_list)',len(i_list))
    
    if len(i_list)==0: break
    i=i_list.pop(0)
    print('i',i)
    try:
        result=check_i_paper(result,i)
    except Exception as e:
        print(e)
        print('error',i)
        i_list.append(i)


 --------------------- 
len(i_list) 3
i 0
 --------------------- 
len(i_list) 2
i 5
 --------------------- 
len(i_list) 1
i 8
 --------------------- 
len(i_list) 0


In [31]:
result=result.sort_values(by='引文總數',ascending=False).reset_index(drop=True)

In [32]:
result.to_pickle(path)